<a href="https://colab.research.google.com/github/mshafi710/kttsdataenrichment/blob/main/club_excel_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reading  multiple excel files and club them into single one.


In [1]:
from google.colab import drive
import os
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
!pip install python-docx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=a28d4549e840e46312e49ba24e44c5af1bc349cf57052f42fe4db0062112fc68
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [2]:
drive.mount('/content/drive')
combined_data = pd.DataFrame()

Mounted at /content/drive


In [3]:
from docx import Document

document = Document()
directory = '/content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files'
tqdm_files  = tqdm_notebook(os.listdir(directory), unit="item", ncols= 1000, desc = " Processing ..")
files_without_kashmiri_column= 0
files_skipped_dueto_error = 0
desired_header_name = 'kashmiri'
for filename in tqdm_files:
    if filename.endswith(".xlsx") or filename.endswith(".xls"):

      file_path = os.path.join(directory, filename)
      try:
          tqdm_files.set_description_str(f"Processing {filename}")
          df = pd.read_excel(file_path)
          # kashmiri_column = df.iloc[:, 0].astype(str)
          matching_columns = [col for col in df.columns if str(col).lower() == desired_header_name.lower()]

          if  matching_columns:
            kashmiri_column = df[matching_columns[0]]
            for value in kashmiri_column:
              document.add_paragraph(str(value))
          else:
            print (' Kashmiri Column not present in {}'.format(file_path))
            files_without_kashmiri_column = files_without_kashmiri_column + 1


      except ValueError:
        print("\nError in reading file {}. Skipping it. \n".format(filename))
        files_skipped_dueto_error = files_skipped_dueto_error + 1
print ( ' Skipped Files: {} \n Files with kashmiri column: {}'. format (  files_skipped_dueto_error, files_without_kashmiri_column))
tqdm(document.save("/content/drive/MyDrive/PhD/Dataset/Text/text_files/corpus/news_corpus.docx"), desc="Saving Resultant file")

 Processing ..:   0%|                                                                                         …

 Kashmiri Column not present in /content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files/fullnews_id_199788_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.xlsx
 Kashmiri Column not present in /content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files/fullnews_id_199888_date_07_25_2.xlsx
 Kashmiri Column not present in /content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files/fullnews_id_199789_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.xlsx
 Kashmiri Column not present in /content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files/fullnews_id_199790_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.xlsx
 Kashmiri Column not present in /content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files/fullnews_id_199791_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.xlsx
 Kashmiri Column not present in /content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files

Saving Resultant file: 0it [00:00, ?it/s]

# Below code was an attempt to process those files for which there is no header information and by the text itself was tried to figure out which column contains urdu and which one contains kashmiri text. Atleast there are 152 files which are header less in addition to 2 files which have reading problem.


In [ ]:
from docx import Document
import re
document = Document()
directory = '/content/drive/MyDrive/PhD/Dataset/Text/text_files/translated_files'
tqdm_files  = tqdm_notebook(os.listdir(directory), unit="item", ncols= 1000, desc = " Processing ..")
files_without_kashmiri_column= 0
files_skipped_dueto_error = 0
desired_header_name = 'kashmiri'
urdu_pattern = re.compile(r'[\u0600-\u06FF\s]+')
kashmiri_pattern = re.compile(r'[\u0620]')
for filename in tqdm_files:
    if filename.endswith(".xlsx") or filename.endswith(".xls"):

      file_path = os.path.join(directory, filename)
      try:
          tqdm_files.set_description_str(f"Processing {filename}")
          df = pd.read_excel(file_path)
          # kashmiri_column = df.iloc[:, 0].astype(str)
          matching_columns = [col for col in df.columns if str(col).lower() == desired_header_name.lower()]
          print(f'processing {filename} ')
          if  matching_columns:
            print("kashmri column present by heading")
          else:
            check_cols(df)
      except ValueError:
          print("\nError in reading file {}. Skipping it. \n".format(filename))
          files_skipped_dueto_error = files_skipped_dueto_error + 1

print ( ' Skipped Files: {} \n Files with kashmiri column: {}'. format (  files_skipped_dueto_error, files_without_kashmiri_column))
tqdm(document.save("/content/drive/MyDrive/PhD/Dataset/Text/text_files/corpus/news_corpus.docx"), desc="Saving Resultant file")


In [5]:
def check_cols(df):
  for column in df.columns:
        contains_urdu = any(re.search(urdu_pattern, str(value)) for value in df[column])
        contains_kashmiri = any(re.search(kashmiri_pattern, str(value)) for value in df[column])

        if contains_urdu and contains_kashmiri:
            print(f"Column {column}: Contains Urdu and Kashmiri text")
        elif contains_urdu:
            print(f"Column {column}: Contains Urdu text")
        elif contains_kashmiri:
            print(f"Column {column}: Contains Kashmiri text")
        else:
            print(f"Column {column}: Does not contain Urdu or Kashmiri text")
